In [1]:
import numpy as np
import os
import cv2

In [8]:
images_path = "../images_and_poses_for_project_assignment/"
images_path = [os.path.join(images_path, imagename) for imagename in os.listdir(images_path) if imagename.endswith(".png")]

In [21]:
A = []
grid_size = (8, 11) #the number of corners on each side

for p in images_path:
    # print(p)
    im = cv2.imread(p)
    return_value, corners = cv2.findChessboardCorners(im, patternSize=grid_size, corners=None)
    corners = corners.squeeze(1)

    if not return_value:
        print(f"Pattern not found for image {p}")

    # CONSTRUCT A

    square_size = 11
    # A = np.empty((0, 9), dtype=float)

    for index, corner in enumerate(corners):
        # complete appropriately the construction of A
        
        # getting the coordinates in pixels
        u_coord = corner[0]
        v_coord = corner[1]
        
        # defining the grid structure of the checkerboard
        grid_size_cv2 = tuple(reversed(grid_size))  # we want (rows, cols), not (cols, rows)
        u_index, v_index = np.unravel_index(index, grid_size_cv2)  # the first corner is at position (0,0), the second (0,1)
        
        # finding the (x,y) coordinates wrt the checkerboard
        x_mm = (u_index) * square_size
        y_mm = (v_index) * square_size
        
        eq_1 = [x_mm, y_mm, 1, 0, 0, 0, -u_coord*x_mm, -u_coord*y_mm, -u_coord]
        eq_2 = [0, 0, 0, x_mm, y_mm, 1, -v_coord*x_mm, -v_coord*y_mm, -v_coord]
        
        A.append(eq_1)
        A.append(eq_2)

# SOLVE A*h=0 for h using Singular Value Decomposition (use the function numpy.linalg.svd: https://numpy.org/doc/2.3/reference/generated/numpy.linalg.svd.html)
A = np.array(A)
U, S, V = np.linalg.svd(A, full_matrices=False) # no padding and faster

# RECOVER the matrix H from the solution h
# V is transposed so the value of H are in the last row
H = V[-1, :].reshape(3,3)
print(H)

[[-5.58566928e-03 -9.16166192e-03  7.92157824e-01]
 [-4.05713329e-03 -6.68900874e-03  6.10170594e-01]
 [-8.77973408e-06 -1.30158037e-05  1.22021407e-03]]


In [22]:
image_path = images_path[0]
im = cv2.imread(image_path)
return_value, corners = cv2.findChessboardCorners(im, patternSize=grid_size, corners=None)
corners = corners.squeeze(1)

In [23]:
_u = float((H[0,:] @ np.array([0,0,1])) / (H[2,:] @ np.array([0,0,1])))
_v = float((H[1,:] @ np.array([0,0,1])) / (H[2,:] @ np.array([0,0,1])))
print(f"Pixel coords of (0,0,1): [{_u:.4f}, {_v:.4f}]")
print(f"Pixel coords expected: {corners[0]}")

Pixel coords of (0,0,1): [649.1958, 500.0521]
Pixel coords expected: [538.10095 506.65207]
